# *ETL realizado sobre los datos de Google*

## Comenzamos con el dataset de las reviews

***importamos librerias a utilizar***

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

***Extraemos los datos***

In [2]:
# Especifica la ruta de la carpeta donde están los archivos JSON
folder_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\Proyecto Final\datasets\Google Maps\review-estados\review-California'

# Obtiene los todos los archivos de la carpeta
files = os.listdir(folder_path)
dataframes = []

# Leer cada archivo JSON línea por línea
for file_name in files:
    if file_name.endswith('.json'):  # Asegurarse de que solo se lean los archivos JSON
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_json(file_path, lines=True)  # Leer el archivo JSON línea por línea
            dataframes.append(df)
        except ValueError as e:
            print(f"Error leyendo {file_name}: {e}")

# Concatenar todos los DataFrames en uno solo
df_reviews_g = pd.concat(dataframes, ignore_index=True)


***Visualizamos la cantidad de registros de este Dataset***

In [3]:
len(df_reviews_g)

2700000

***Transformaciones iniciales determinadas por el EDA***

In [4]:
# Eliminar la columna 'pics' ya que no la necesitamos para este analisis
df_reviews_g.drop(columns=['pics'], inplace=True)

# Convertir los diccionarios en la columna 'resp' a cadenas de texto
df_reviews_g['resp'] = df_reviews_g['resp'].apply(lambda x: str(x) if isinstance(x, dict) else x)

# Convertir la columna 'time' a formato de fecha
df_reviews_g['time'] = pd.to_datetime(df_reviews_g['time'], unit='ms')

# Eliminar duplicados
df_reviews_g.drop_duplicates(inplace=True)

**En un analisis de primera instancia, determinamos tambien, que la funcionalidad de hacer una reseña en un local existente en Google Maps se agrego en el año 2007. Por lo tanto todas las reseñas previas a este año las consideramos erroneas y son descartadas.**

In [5]:
# Asegurarse de que la columna 'time' esté en formato datetime
df_reviews_g['time'] = pd.to_datetime(df_reviews_g['time'], errors='coerce')

# Detectar outliers en 'time' (fechas fuera del rango)
min_date = pd.to_datetime('2007-06-01')
max_date = pd.to_datetime('today')

# Filtrar las filas que tienen fechas fuera de este rango
outliers_time = df_reviews_g[(df_reviews_g['time'] < min_date) | (df_reviews_g['time'] > max_date)]

# Eliminamos estos outliers
df_reviews_g = df_reviews_g[(df_reviews_g['time'] >= min_date) & (df_reviews_g['time'] <= max_date)]

***Eliminamos la columna 'resp' ya que no la necesitamos para este analisis***

In [6]:
df_reviews_g = df_reviews_g.drop(columns=['resp'])

***Reemplazamos valores nulos en la columna 'review_text' con 'Sin Reseña'***

In [7]:
df_reviews_g['text'].fillna('Sin Reseña', inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13908\1590490907.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_reviews_g['text'].fillna('Sin Reseña', inplace=True)


***Luego de estas transformaciones, revisamos nuevamente la cantidad de registros***

In [8]:
len(df_reviews_g)

2624574

## Pasamos al dataset sobre los Metadatos

***importamos librerias a utilizar***

In [9]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

***Extraemos los datos***

In [10]:
# Ruta del archivo .pkl
folder_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\Proyecto Final\datasets\Google Maps\metadata-sitios'

# Obtiene los todos los archivos de la carpeta
files = os.listdir(folder_path)
dataframes = []

# Leer cada archivo JSON línea por línea
for file_name in files:
    if file_name.endswith('.json'):  # Asegurarse de que solo se lean los archivos JSON
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_json(file_path, lines=True)  # Leer el archivo JSON línea por línea
            dataframes.append(df)
        except ValueError as e:
            print(f"Error leyendo {file_name}: {e}")

# Concatenar todos los DataFrames en uno solo
df_business_google = pd.concat(dataframes, ignore_index=True)

***Verificamos la cantidad de registros***

In [11]:
len(df_business_google)

3025011

***Desanidamos Misc***

In [12]:
# Desanidar la columna 'MISC' que contiene JSON
misc_expanded = pd.json_normalize(df_business_google['MISC'])

# Combinar las columnas desanidadas con el DataFrame original
df_business_google = pd.concat([df_business_google, misc_expanded], axis=1)

# Eliminar la columna original 'MISC'
df_business_google = df_business_google.drop(columns=['MISC'])

# Convertir listas en las nuevas columnas a cadenas separadas por comas
df_business_google = df_business_google.applymap(
    lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x
)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13908\2198222021.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_business_google = df_business_google.applymap(


***Hacemos transformaciones***

In [13]:
# Eliminar las columnas 'price', 'hours' y 'url'
df_business_google = df_business_google.drop(columns=['price', 'hours', 'url'])

# Eliminar filas duplicadas completamente idénticas
df_business_google = df_business_google.drop_duplicates()

# Convertir la columna 'state' a tipo texto (string)
df_business_google['state'] = df_business_google['state'].astype(str)

# Eliminar filas donde 'state' es igual a 'Permanently closed'
df_business_google = df_business_google[df_business_google['state'] != 'Permanently closed']

# Rellenamos los valores nulos con el texto "Valores faltantes"
df_business_google['category'].fillna('Valores Faltantes', inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13908\2017661347.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_business_google['category'].fillna('Valores Faltantes', inplace=True)


***Verificamos la cantidad de registros luego de los cambios realizados***

In [14]:
len(df_business_google)

2809262

***Filtramos para dejar solo los registros con categorias que consideramos dentro de la gastronomia***

In [15]:
# Definimos palabras clave a buscar luego en las categorias
palabra = r'\b(restaurant|cafe|delivery|diner|bistro|takeout|bar|pub|grill|pizzeria|coffee|bakery|food|eatery|sandwich|snack)\b'

# Filtrar las categorías coincidentes
df_business_google = df_business_google[df_business_google['category'].str.contains(palabra, case=False, regex=True, na=False)]

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13908\3655407268.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_business_google = df_business_google[df_business_google['category'].str.contains(palabra, case=False, regex=True, na=False)]


***Generamos una columna "clacificacion" a partir de la categoria***

In [16]:
clasificacion = {
    'Dining Venue': ['Restaurant'],
    'Quick Service': ['Fast food restaurant', 'Pizza restaurant', 'Pizza Takeout'],
    'Takeout & Delivery': ['Takeout Restaurant', 'Delivery Restaurant'],
    'Mexican Dining': ['Mexican restaurant', 'Taco restaurant'],
    'American Dining': ['American restaurant', 'Burger restaurant'],
    'Sandwich Bar': ['Sandwich shop'],
    'Italian Dining': ['Italian restaurant'],
    'Chinese Dining': ['Chinese restaurant'],
    'Café & Coffee': ['Coffee shop', 'Cafe'],
    'Seafood Dining': ['Seafood restaurant'],
    'Barbecue Dining': ['Barbecue restaurant'],
    'Asian Fusion': ['Asian restaurant', 'Sushi restaurant', 'Japanese restaurant', 'Thai restaurant'],
    'Chicken House': ['Chicken restaurant', 'Chicken wings restaurant'],
    'Bakery Shop': ['Bakery', 'Bakery shop'],
    'Ice Cream Parlor': ['Ice cream shop'],
    'Indian Dining': ['Indian restaurant'],
    'Latin American Dining': ['Latin American restaurant'],
    'Juice Bar': ['Juice shop'],
    'Specialty Catering': ['Caterer'],
    'Vegetarian Dining': ['Vegetarian restaurant'],
    'Vietnamese Dining': ['Vietnamese Restaurant'],
    'Health Foods': ['Health food store', 'Health Food', 'Green Food']
}


def assign_group(category):
    """
    
    """
    if pd.isna(category):  # Si la categoría es Nula
        return 'Other'
    for key, values in clasificacion.items():
        for value in values:
            if value.lower() in category.lower():
                return key
    return 'Other'

df_business_google['clasificacion'] = df_business_google['category'].map(assign_group)

In [17]:
df_business_google = df_business_google[df_business_google['clasificacion'] != 'Other']

### Filtramos los datos mediante el uso de la columna gmap_id (compartida por ambos datasets) para asi dejar tan solo los datos de california en el Dataframe con los datos de los locales.

In [18]:
# Filtrar los registros en df_business_google que coincidan en la columna 'gmap_id'
df_business_google = df_business_google[df_business_google['gmap_id'].isin(df_reviews_g['gmap_id'])]

***Verificamos nuevamente la cantidad de registros para comparar***

In [19]:
len(df_business_google)

7923

### Volvemos a transformar el dataset "Reviews"

A partir del dataframe df_reviews_g sacamos la columna gmap_id con valores únicos. De esa forma, esa lista permitio filtrar los registros pertenecientes al estado de California en el dataset metadatos de google.
Una vez que tenemos aquellos registros definidos, en los que se ha delimitado el dataset al rubro gastronomico y afines, corresponde ahora volver a filtrar este dataset para que solo queden a disposición los registros del estado de california vinculados al sector gastronomico, lo que haremos a continuación.

In [20]:
# Filtrar los registros en df_reviews_g que coincidan en la columna 'gmap_id'
df_reviews_google = df_reviews_g[df_reviews_g['gmap_id'].isin(df_business_google['gmap_id'])]

In [21]:
df_reviews_google.head()

,user_id,name,time,rating,text,gmap_id
0,1.089912e+20,Song Ro,2021-01-06 05:12:07.056,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-02-09 05:47:28.663,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-03-08 05:04:42.296,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,2019-03-07 05:56:56.355,5,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,2017-05-16 05:01:41.933,5,Great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49


## Guardamos los Dataframes finales en archivos .parquet

In [22]:
df_reviews_google.to_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\reviews_google.parquet')
df_business_google.to_parquet(r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\repo\epicurean_project\DATA\data parcial\locales_google.parquet')